# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [1]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
customer_df1 = pd.read_csv(url1)

pd.options.display.float_format = '{:,.2f}'.format

customer_df1.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,"4,809.22",No,Basic,College,2/18/11,Employed,M,...,0.00,9,Corporate Auto,Corporate L3,Offer3,Agent,292.80,Four-Door Car,Medsize,NaN
1,1,KX64629,California,"2,228.53",No,Basic,College,1/18/11,Unemployed,F,...,0.00,1,Personal Auto,Personal L3,Offer4,Call Center,744.92,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,"14,947.92",No,Basic,Bachelor,2/10/11,Employed,M,...,0.00,2,Personal Auto,Personal L3,Offer3,Call Center,480.00,SUV,Medsize,A
3,3,XL78013,Oregon,"22,332.44",Yes,Extended,College,1/11/11,Employed,M,...,0.00,2,Corporate Auto,Corporate L3,Offer2,Branch,484.01,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,"9,025.07",No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.93,Four-Door Car,Medsize,NaN


In [2]:

customer_df1.isnull().sum()

Unnamed: 0                          0
Customer                            0
State                             631
Customer Lifetime Value             0
Response                          631
Coverage                            0
Education                           0
Effective To Date                   0
EmploymentStatus                    0
Gender                              0
Income                              0
Location Code                       0
Marital Status                      0
Monthly Premium Auto                0
Months Since Last Claim           633
Months Since Policy Inception       0
Number of Open Complaints         633
Number of Policies                  0
Policy Type                         0
Policy                              0
Renew Offer Type                    0
Sales Channel                       0
Total Claim Amount                  0
Vehicle Class                     622
Vehicle Size                      622
Vehicle Type                     5482
dtype: int64

In [3]:
from typing import Optional, Iterable, Dict
def standardize_text(
    df: pd.DataFrame,
    cols: Optional[Iterable[str]] = None,
    lower: bool = True,
    strip: bool = True,
    normalize_columns: bool = True,
    normalize_colnames: Optional[Dict[str, str]] = None,
    numeric_map: Optional[Dict[str, str]] = None
) -> pd.DataFrame:
# 0) Normalizar nombres de columnas
    if normalize_columns:
        df.columns = (
            df.columns
              .str.strip()
              .str.lower()
              .str.replace(" ", "_", regex=False)
        )
# 0.1) Renombrados específicos (si se pasan)
    if normalize_colnames:
        df = df.rename(columns={k.lower(): v.lower() for k, v in normalize_colnames.items()})
# 1) Normalizar texto en columnas string/object
    if cols is None:
        cols = df.select_dtypes(include=["object", "string"]).columns
    for c in cols:
        s = df[c].astype("string")
        if strip:
            s = s.str.strip()
        if lower:
            s = s.str.lower()
        df[c] = s
# 2) Normalizar tipos numéricos
    if numeric_map:
        for col, dtype in numeric_map.items():
            if col in df.columns:
                ser = df[col].astype("string").str.strip().str.replace(",", ".", regex=False)
                ser = pd.to_numeric(ser, errors="coerce")
                if dtype == "Int64":
                    df[col] = ser.astype("Int64")
                else:
                    df[col] = ser.astype(float)
                df[col] = ser.astype(float)
                
    return df

normalize_map = {'st' : 'state'}
numeric_map = {
        "customer_lifetime_value": float,
        "income": float,
        "monthly_premium_auto": float,
        "number_of_open_complaints": "Int64",
        "total_claim_amount": float,
    }


customer_df1 = standardize_text(customer_df1, normalize_colnames=normalize_map, numeric_map=numeric_map)

In [4]:
for col in customer_df1:
    print(f'La columna {col.upper()} contiene los valores:', customer_df1[col].unique())

La columna UNNAMED:_0 contiene los valores: [    0     1     2 ... 10907 10908 10909]
La columna CUSTOMER contiene los valores: <StringArray>
['dk49336', 'kx64629', 'lz68649', 'xl78013', 'qa50777', 'ho81102', 'iw72280',
 'ih64929', 'fm55990', 'qx31376',
 ...
 'mm70762', 'so95845', 'vn73653', 'ry92254', 'pp30874', 'su71163', 'qi63521',
 'kx53892', 'tl39050', 'wa60547']
Length: 9134, dtype: string
La columna STATE contiene los valores: <StringArray>
['arizona', 'california', 'washington', 'oregon', <NA>, 'nevada']
Length: 6, dtype: string
La columna CUSTOMER_LIFETIME_VALUE contiene los valores: [ 4809.21696   2228.525238 14947.9173   ...  5259.444853 23893.3041
 11971.97765 ]
La columna RESPONSE contiene los valores: <StringArray>
['no', 'yes', <NA>]
Length: 3, dtype: string
La columna COVERAGE contiene los valores: <StringArray>
['basic', 'extended', 'premium']
Length: 3, dtype: string
La columna EDUCATION contiene los valores: <StringArray>
['college', 'bachelor', 'high school or below

In [5]:
customer_df1.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,dk49336,arizona,"4,809.22",no,basic,college,2/18/11,employed,m,...,0.00,9,corporate auto,corporate l3,offer3,agent,292.80,four-door car,medsize,<NA>
1,1,kx64629,california,"2,228.53",no,basic,college,1/18/11,unemployed,f,...,0.00,1,personal auto,personal l3,offer4,call center,744.92,four-door car,medsize,<NA>
2,2,lz68649,washington,"14,947.92",no,basic,bachelor,2/10/11,employed,m,...,0.00,2,personal auto,personal l3,offer3,call center,480.00,suv,medsize,a
3,3,xl78013,oregon,"22,332.44",yes,extended,college,1/11/11,employed,m,...,0.00,2,corporate auto,corporate l3,offer2,branch,484.01,four-door car,medsize,a
4,4,qa50777,oregon,"9,025.07",no,premium,bachelor,1/17/11,medical leave,f,...,NaN,7,personal auto,personal l2,offer1,branch,707.93,four-door car,medsize,<NA>


In [6]:
customer_df1.describe()

,unnamed:_0,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
count,"10,910.00","10,910.00","10,910.00","10,910.00","10,277.00","10,910.00","10,277.00","10,910.00","10,910.00"
mean,"5,454.50","8,018.24","37,536.28",93.20,15.15,48.09,0.38,2.98,434.89
std,"3,149.59","6,885.08","30,359.20",34.44,10.08,27.94,0.91,2.40,292.18
min,0.00,"1,898.01",0.00,61.00,0.00,0.00,0.00,1.00,0.10
25%,"2,727.25","4,014.45",0.00,68.00,6.00,24.00,0.00,1.00,271.08
50%,"5,454.50","5,771.15","33,813.50",83.00,14.00,48.00,0.00,2.00,382.56
75%,"8,181.75","8,992.78","62,250.75",109.00,23.00,71.00,0.00,4.00,547.20
max,"10,909.00","83,325.38","99,981.00",298.00,35.00,99.00,5.00,9.00,"2,893.24"


In [ ]:
# 1 Create a new DataFrame that only includes customers who:
#  - have a **low total_claim_amount** (e.g., below $1,000),
#  - have a response "Yes" to the last marketing campaign.
filtered_df = customer_df1[
    (customer_df1['total_claim_amount'] < 1000) & 
    (customer_df1['response'] == 'yes')
]
filtered_df.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
3,3,xl78013,oregon,"22,332.44",yes,extended,college,1/11/11,employed,m,...,0.00,2,corporate auto,corporate l3,offer2,branch,484.01,four-door car,medsize,a
8,8,fm55990,california,"5,989.77",yes,premium,college,1/19/11,employed,m,...,0.00,1,personal auto,personal l1,offer2,branch,739.20,sports car,medsize,<NA>
15,15,cw49887,california,"4,626.80",yes,basic,master,1/16/11,employed,f,...,0.00,1,special auto,special l1,offer2,branch,547.20,suv,medsize,<NA>
19,19,nj54277,california,"3,746.75",yes,extended,college,2/26/11,employed,f,...,1.00,1,personal auto,personal l2,offer2,call center,19.58,two-door car,large,a
27,27,mq68407,oregon,"4,376.36",yes,premium,bachelor,2/28/11,employed,f,...,0.00,1,personal auto,personal l3,offer2,agent,60.04,four-door car,medsize,<NA>


In [8]:
filtered_df.describe()

,unnamed:_0,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
count,"1,399.00","1,399.00","1,399.00","1,399.00","1,324.00","1,399.00","1,324.00","1,399.00","1,399.00"
mean,"5,369.29","7,709.17","38,483.35",88.72,14.64,47.42,0.36,2.89,412.13
std,"3,138.69","6,261.53","28,191.66",22.64,9.78,26.80,0.93,2.42,180.46
min,3.00,"2,004.35",0.00,61.00,0.00,0.00,0.00,1.00,7.35
25%,"2,627.00","3,936.41","18,495.00",67.00,6.00,25.00,0.00,1.00,312.00
50%,"5,423.00","5,548.03","33,190.00",84.00,14.00,50.00,0.00,2.00,398.50
75%,"7,933.50","9,031.21","60,920.00",107.00,22.00,68.00,0.00,3.00,528.20
max,"10,897.00","41,787.90","99,845.00",154.00,35.00,99.00,5.00,9.00,960.12


In [ ]:
#2 Using the original Dataframe, analyze: the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` 
# for customers who responded "Yes", and
avg_values = customer_df1[customer_df1['response'] == 'yes'].groupby(['policy_type', 'gender']).agg({
    'monthly_premium_auto': 'mean',
    'customer_lifetime_value': 'mean', 
}).reset_index()
avg_values.head()


,policy_type,gender,monthly_premium_auto,customer_lifetime_value
0,corporate auto,f,94.30,"7,712.63"
1,corporate auto,m,92.19,"7,944.47"
2,personal auto,f,99.00,"8,339.79"
3,personal auto,m,91.09,"7,448.38"
4,special auto,f,92.31,"7,691.58"


# compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.


In [ ]:
#3. Analyze the total number of customers who have policies in each state, and then filter the results to only 
# include states where there are more than 500 customers.
customer_counts = customer_df1.groupby('state').agg({
    'customer': 'count'
}).reset_index().rename(columns={'customer': 'total_customers'})

filtered_states = customer_counts[customer_counts['total_customers'] > 500]     
filtered_states.head()

,state,total_customers
0,arizona,1937
1,california,3552
2,nevada,993
3,oregon,2909
4,washington,888


In [37]:
# 4 Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
lifetime_stats = customer_df1.groupby(['education', 'gender']).agg({
    'customer_lifetime_value': ['max', 'min', 'median']
}).reset_index()
lifetime_stats.head()

education gender customer_lifetime_value                  
                                       max      min   median
0  bachelor      f               73,225.96 1,904.00 5,640.51
1  bachelor      m               67,907.27 1,898.01 5,548.03
2   college      f               61,850.19 1,898.68 5,623.61
3   college      m               61,134.68 1,918.12 6,005.85
4    doctor      f               44,856.11 2,395.57 5,332.46

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [38]:
#5. The marketing team wants to analyze the number of policies sold by state and month. 
# First, extract month from effective_to_date
customer_df1['month'] = pd.to_datetime(customer_df1['effective_to_date']).dt.month
policy_sales = customer_df1.pivot_table(index='state', columns='month', values='policy', aggfunc='count', fill_value=0)
policy_sales.head()

C:\Users\marta\AppData\Local\Temp\ipykernel_21664\785968352.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  customer_df1['month'] = pd.to_datetime(customer_df1['effective_to_date']).dt.month


month,1,2
state,,
arizona,1008,929
california,1918,1634
nevada,551,442
oregon,1565,1344
washington,463,425


In [39]:
6.  #Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.
top_states = policy_sales.sum(axis=1).nlargest(3).index
top_policy_sales = policy_sales.loc[top_states]
top_policy_sales.head()


month,1,2
state,,
california,1918,1634
oregon,1565,1344
arizona,1008,929


In [40]:
#7 The marketing team wants to analyze the effect of different marketing channels on the customer response rate.
response_rates = customer_df1.pivot_table(index='sales_channel', columns='response', values='customer', aggfunc='count', fill_value=0)
response_rates['response_rate'] = response_rates['yes'] / (response_rates['yes'] + response_rates['no'])
response_rates.head()

response,no,yes,response_rate
sales_channel,,,
agent,3148,742,0.19
branch,2539,326,0.11
call center,1792,221,0.11
web,1334,177,0.12
